## Reduction operation: the sum of the numbers in the range [0, value)

In [3]:
import numpy as np
import sys

def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Secuencial

value = int(sys.argv[1])

X = np.random.rand(value)

# Para imprimir los pimeros valores del array

# print(X[0:12])

# Utilizando las operaciones mágicas de ipython

tiempo = %timeit -r 2 -o -q reduc_operation(X)

print("Time taken by reduction operation using a function:", tiempo)


print(f"And the result of the sum of numbers in the range [0, value) is: {reduc_operation(X)}\n")


# Utilizando numpy.sum()

tiempo = %timeit -r 2 -o -q np.sum(X)

print("Time taken by reduction operation using numpy.sum():", tiempo)

print("Now, the result using numpy.sum():", np.sum(X),"\n ")


# Utilizando numpy.ndarray.sum()

tiempo= %timeit -r 2 -o -q X.sum()

print("Time taken by reduction operation using numpy.ndarray.sum():", tiempo)

print("Now, the result using numpy.ndarray.sum():", X.sum())




Time taken by reduction operation using a function: 46.8 ms ± 43.5 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
And the result of the sum of numbers in the range [0, value) is: 24961.09920301303

Time taken by reduction operation using numpy.sum(): 98 µs ± 884 ns per loop (mean ± std. dev. of 2 runs, 10,000 loops each)
Now, the result using numpy.sum(): 24961.099203013076 
 
Time taken by reduction operation using numpy.ndarray.sum(): 74.2 µs ± 171 ns per loop (mean ± std. dev. of 2 runs, 10,000 loops each)
Now, the result using numpy.ndarray.sum(): 24961.099203013076


### 3.2. Python HPC: Paralelismo con multiprocessing y Numba

##### EJERCICIO A

La ejecución secuencial no aprovecha múltiples núcleos de la CPU. Para arrays de mayor tamaño, esto puede ser lento. Con este ejercicio lo que se está haciendo es paralelizar la operación de reducción dividiendo el array en subarrays y procesándolos en paralelo gracias a la introducción de la librería multiprocessing.

Primero tendremos que usar el bojeto Pool de esta librería para poder gestionar varios procesos y dividir así el trabajo.
Luego tendremos que dividir el array original en subarrays de distintos tamaños.
Por último, aplicamos la función reduc_operation.

In [4]:
# Importamos Pool de la librería multiprocessing
from multiprocessing import Pool

# Establecemos el main para asegurar que el código dentro de este bloque solo se ejecute cuando el script se ejectua directamente
if __name__ == "__main__":
    
    # Con un solo proceso: tamaño del array original de [0, value]
    with Pool(1) as pool:
        t1 = %timeit -r 2 -o -q pool.map(reduc_operation, [X])
        print("El tiempo con un solo proceso es:", t1)
        
    # Con dos procesos: 2 arrays de tamaños [0, int(value/2)] y [int(value/2), value]
    with Pool(2) as pool:
        s_array2 = [X[0:int(value/2)], X[int(value/2):value]]
        t2 = %timeit -r 2 -o -q pool.map(reduc_operation, s_array2)
        print("El tiempo con dos procesos es:", t2)
              
    # Con cuatro procesos: 4 arrays de tamaños [0, int(value/4)], [int(value/4), int(value/2)], [int(value/2), int(3*value/4)] y [int(3*value/4), value]
    with Pool(4) as pool:
        s_array4 = [
            X[0:int(value/4)],
            X[int(value/4):int(value/2)],
            X[int(value/2):int(3*value/4)],
            X[int(3*value/4):value]
        ]

        t3 = %timeit -r 2 -o -q pool.map(reduc_operation, s_array4)
        print("El tiempo con cuatro procesos es:", t3)


El tiempo con un solo proceso es: 50.2 ms ± 220 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
El tiempo con dos procesos es: 29.4 ms ± 101 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
El tiempo con cuatro procesos es: 31.9 ms ± 1.68 ms per loop (mean ± std. dev. of 2 runs, 10 loops each)


Cuando el array es pequeño, como en este caso con 50,000 elementos, el tiempo extra que el programa necesita para dividir el trabajo, crear más procesos, coordinarlos y juntar los resultados (overhead) puede ser mayor que el beneficio de usar más procesos. Por eso, con 2 procesos es más eficiente, ya que se logra un buen balance entre dividir el trabajo y minimizar ese tiempo adicional. Usar 4 procesos añade más overhead, lo que termina siendo menos eficiente. Aunque luego veremos, cuando corramos el programa con el gestor de colas, que esto puede variar.

##### EJERCICIO B

- @njit es un decorador de Numba que básicamente transforma una función de Python en código de máquina altamente optimizado. Esto quiere decir que la función se compila la primera vez que se llama, eliminando la sobrecarga de interpretación de Python.

- prange es una versión paralelizada de range, que está diseñada para poder trabajar con el @njit. Permite dividir un bucle entre múltiples núcleos de la CPU, haciendo que las iteraciones se ejecuten simultáneamente.

Con este ejercicio lo que estamos haciendo es optimizar la función reduc_operation usando Numba y paralelismo. Numba va a compilar la función directamente en código de máquina mediante @njit y prange va a dividir automáticamente el bucle entre los núcleos disponibles.

In [5]:
# Importamos primero las librerías
import numpy as np
from numba import njit, prange

# Versión optimizada con @njit
@njit
def reduc_operation_njit(A):
    "Función de reducción optimizada con @njit"
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Versión paralela con @njit(parallel=True)
@njit(parallel=True)
def reduc_operation_njit_prange(A):
    """Función de reducción optimizada yu paralelizada con @njit y prange"""
    s = 0
    for i in prange(A.size):  # prange para paralelismo
        s += A[i]
    return s


# Comparamos los tiempos
tiempo_o = %timeit -r 2 -o -q reduc_operation(X)
print("Tiempo que usa la función original:", tiempo)
tiempo_njit = %timeit -r 2 -o -q reduc_operation_njit(X)
print("Tiempo que usa la función con el decorador njit:", tiempo_njit)
tiempo_njit_p = %timeit -r 2 -o -q reduc_operation_njit_prange(X)
print("Tiempo que usa la función con el decorador njit y paralelizada:", tiempo_njit_p)

Tiempo que usa la función original: 74.2 µs ± 171 ns per loop (mean ± std. dev. of 2 runs, 10,000 loops each)
Tiempo que usa la función con el decorador njit: 214 µs ± 4.71 µs per loop (mean ± std. dev. of 2 runs, 1 loop each)
Tiempo que usa la función con el decorador njit y paralelizada: 41.2 ms ± 1.56 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


##### EJERCICIO C

El ejercicio C consiste en realizar el shell script para ejecutar el programa con sbatch.
Se ha añadido a GitHub.

##### EJERCICIO D

###### Resultados de la ejecución del script de jupyter en la cola pascal:

Ejecutando el notebook de jupyter con 50000000 elementos

Time taken by reduction operation using a function: 45.4 s ± 74.6 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 25000257.207552828

Time taken by reduction operation using numpy.sum(): 60.1 ms ± 199 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
Now, the result using numpy.sum(): 25000257.207554013

Time taken by reduction operation using numpy.ndarray.sum(): 60.3 ms ± 345 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
Now, the result using numpy.ndarray.sum(): 25000257.207554013

El tiempo con un solo proceso es: 48 s ± 5.77 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

El tiempo con dos procesos es: 25.7 s ± 25.3 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

El tiempo con cuatro procesos es: 13.3 s ± 1.04 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


Tiempo que usa la función original: 60.3 ms ± 345 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)

Tiempo que usa la función con el decorador njit: 207 ms ± 44.8 µs per loop (mean ± std. dev. of 2 runs, 1 loop each)

Tiempo que usa la función con el decorador njit y paralelizada: 17.9 ms ± 5.11 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


##### Conclusiones

Cuando se usa el paquete multiprocessing, se observa una reducción significativa del tiempo a medida que se aumenta el número de procesos. Por ejemplo, al duplicar el número de procesos de 2 a 4, el tiempo se reduce aproximadamente a la mitad. Esto demuestra que multiprocessing es eficiente al dividir el trabajo en múltiples procesos, aunque con el costo de un overhead asociado a la creación y gestión de dichos procesos. Esto lo hace útil, pero no necesariamente óptimo para tareas que requieren alto grado de paralelismo.

Por otro lado, con el decorador njit de Numba, el tiempo de ejecución disminuye drásticamente, pasando de segundos a milisegundos. Esto se debe a que Numba compila la función directamente en código máquina, eliminando la sobrecarga del intérprete de Python. Además, al usar njit(parallel=True), el tiempo de ejecución se reduce aún más, logrando una mejora aproximada de 12 veces en comparación con la versión secuencial optimizada con njit. Esto demuestra cómo Numba puede aprovechar múltiples núcleos de la CPU de manera eficiente y acelerar significativamente tareas intensivas en cálculos.

En conclusión, aunque multiprocessing ofrece beneficios al dividir tareas en múltiples procesos, Numba resulta ser una solución más eficiente al optimizar directamente el código y habilitar el paralelismo. Para operaciones numéricas intensivas y grandes volúmenes de datos, Numba con paralelismo es una excelente opción en sistemas sin GPU.